In [59]:
import cirq
import numpy as np
from typing import Any, Tuple, Optional

In [60]:
class QuditXGate(cirq.Gate):
    def __init__(self, d: int):
        super().__init__()
        self.d = d

    def _qid_shape_(self) -> Tuple[int, ...]:
        return (self.d, )

    def _unitary_(self) -> np.ndarray:
        X = np.zeros((self.d, self.d), dtype=complex)
        for i in range(self.d):
            X[i, (i + 1) % self.d] = 1
        return X

    def _circuit_diagram_info_(self, args: cirq.CircuitDiagramInfoArgs) -> cirq.CircuitDiagramInfo:
        return cirq.CircuitDiagramInfo(wire_symbols=('X',))


In [61]:
class QuditHGate(cirq.Gate):
    def __init__(self, d: int):
        super().__init__()
        self.d = d
        self.omega = np.exp(2j * np.pi / self.d)

    def _qid_shape_(self) -> Tuple[int, ...]:
        return (self.d, )

    def _unitary_(self) -> np.ndarray:
        H = np.array([[self.omega**(k * m) for m in range(self.d)] for k in range(self.d)])
        H /= np.sqrt(self.d)
        return H

    def _circuit_diagram_info_(self, args: cirq.CircuitDiagramInfoArgs) -> cirq.CircuitDiagramInfo:
        return cirq.CircuitDiagramInfo(wire_symbols=('H',))


In [62]:
class QuditZGate(cirq.Gate):
    def __init__(self, d: int):
        super().__init__()
        self.d = d
        self.omega = np.exp(2j * np.pi / self.d)

    def _qid_shape_(self) -> Tuple[int, ...]:
        return (self.d, )

    def _unitary_(self) -> np.ndarray:
        Z = np.diag([self.omega**k for k in range(self.d)])
        return Z

    def _circuit_diagram_info_(self, args: cirq.CircuitDiagramInfoArgs) -> cirq.CircuitDiagramInfo:
        return cirq.CircuitDiagramInfo(wire_symbols=('Z',))


In [63]:
class QuditCNOTGate(cirq.Gate):
    def __init__(self, d: int):
        super().__init__()
        self.d = d

    def _qid_shape_(self) -> Tuple[int, ...]:
        return (self.d, self.d)

    def _unitary_(self) -> np.ndarray:
        size = self.d ** 2
        CNOT = np.zeros((size, size), dtype=complex)
        for a in range(self.d):
            for b in range(self.d):
                control = a
                target = (b + a) % self.d
                row = a * self.d + b
                col = a * self.d + target
                CNOT[row, col] = 1
        return CNOT

    def _circuit_diagram_info_(self, args: cirq.CircuitDiagramInfoArgs) -> cirq.CircuitDiagramInfo:
        return cirq.CircuitDiagramInfo(wire_symbols=('C', 'X'))


In [64]:
def qudit_measure(qudit: cirq.Qid, key: str) -> cirq.Operation:
    return cirq.measure(qudit, key=key)


In [65]:
d = 4  

qudit0 = cirq.LineQid(0, dimension=d)
qudit1 = cirq.LineQid(1, dimension=d)

qudit_x = QuditXGate(d)
qudit_z = QuditZGate(d)
qudit_h = QuditHGate(d)
qudit_cnot = QuditCNOTGate(d)

In [66]:
qudit0

cirq.LineQid(0, dimension=4)

In [67]:
circuit = cirq.Circuit()

circuit.append(qudit_h.on(qudit0))
circuit.append(qudit_cnot.on(qudit0, qudit1))
circuit.append(qudit_x.on(qudit1))
circuit.append(qudit_z.on(qudit0))
circuit.append(qudit_measure(qudit0, key='m0'))
circuit.append(qudit_measure(qudit1, key='m1'))

print("Circuit:")
print(circuit)

Circuit:
0 (d=4): ───H───C───Z───M('m0')───
                │
1 (d=4): ───────X───X───M('m1')───


In [68]:
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=10)

m0_results = result.measurements['m0'].flatten()
m1_results = result.measurements['m1'].flatten()

print("m0 results:", m0_results)
print("m1 results:", m1_results)

m0 results: [3 2 2 3 1 2 2 0 0 0]
m1 results: [0 1 1 0 2 1 1 3 3 3]


# Quantum Teleportation:

In [70]:
message = cirq.LineQid(0, dimension=d)
alice = cirq.LineQid(1, dimension=d)
bob = cirq.LineQid(2, dimension=d)


circuit = cirq.Circuit()
